# 1. Base Model
---

In this notebook, we will prepare dataset and build a base model.

## Import Libraries

In [1]:
from keras import preprocessing, Input, Model, regularizers, optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.models import load_model
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.utils import shuffle

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import sys
sys.path.append("../src")
from utils import destination, save_models

Using TensorFlow backend.


## Create the Target Matrix
Form ```tag_list.txt``` and ```data_list.txt``` in HARRISON dataset, we create a target matrix as a DataFrame.

In [2]:
# Read files
filename = pd.read_csv("../HARRISON/data_list.txt", names=["filename"], header=None)
hashtag = pd.read_csv("../HARRISON/tag_list.txt", names=["labels"], header=None)

# Convert filenames from "instagram_dataset/xxx/yyy.jpg" to "xxx_yyy.jpg"
filename["filename"] = filename["filename"].apply(lambda x: "_".join(x.split("/")[1:]))

# Concatenate filname and labels
target = pd.concat([filename, hashtag], axis=1)
target.head(5)

,filename,labels
0,sea_image_50.jpg,sea instapic instagram trip travel
1,sea_image_1284.jpg,sea
2,sea_image_1122.jpg,sea love
3,sea_image_1679.jpg,beach sea trip island japan
4,sea_image_1736.jpg,sun sand sea sky friend beach thailand trip ad...


In [3]:
# Use vectorizer to generate a one-hot encoding
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(target["labels"])
columns = vectorizer.get_feature_names()
X_df = pd.DataFrame(X.toarray(), columns=columns)

# Combine hashtag encodings with file names
target = pd.concat([target, X_df], axis=1)
target = shuffle(target, random_state=42)

num_images = target.shape[0]
num_hashtags = X_df.shape[1]

## Prepare Images

In [4]:
IMAGE_DIR = "../HARRISON/images"

# Instanciate a data generator
datagen = ImageDataGenerator(rescale=1./255.)

# Use these values to split data into train, val, and test
train_idx = int(target.shape[0] * .70)
val_idx = int(target.shape[0] * .90)

# Create train data generator
train_generator = datagen.flow_from_dataframe(
        dataframe=target[:train_idx],
        directory=IMAGE_DIR,
        x_col="filename",
        y_col=columns,
        batch_size=32,
        seed=42,
        shuffle=True,
        class_mode="raw",
        target_size=(100,100)
    )

# Create val data generator
valid_generator = datagen.flow_from_dataframe(
        dataframe=target[train_idx:val_idx],
        directory=IMAGE_DIR,
        x_col="filename",
        y_col=columns,
        batch_size=32,
        seed=42,
        shuffle=True,
        class_mode="raw",
        target_size=(100,100)
    )

# Create test data generator
test_generator = datagen.flow_from_dataframe(
        dataframe=target[val_idx:],
        directory=IMAGE_DIR,
        x_col="filename",
        batch_size=1,
        seed=42,
        shuffle=False,
        class_mode=None,
        target_size=(100,100)
    )

Found 40168 validated image filenames.
Found 11476 validated image filenames.
Found 5739 validated image filenames.


## Build a Model

In [5]:
# Define model
inp = Input(shape=(100,100,3))
x = Conv2D(32, (3, 3), padding='same')(inp)
x = Activation('relu')(x)
x = Conv2D(32, (3, 3))(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)
x = Conv2D(64, (3, 3), padding='same')(x)
x = Activation('relu')(x)
x = Conv2D(64, (3, 3))(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)
x = Flatten()(x)
x = Dense(512)(x)
x = Activation('relu')(x)
x = Dropout(0.5)(x)
output = []
for i in range(num_hashtags):
    output.append(Dense(1, activation='sigmoid')(x))

model = Model(inp,output)

# Compile model
model.compile(optimizers.rmsprop(
    lr=0.0001,
    decay=1e-6),
    loss = ["binary_crossentropy" for i in range(num_hashtags)],
    metrics=["accuracy"])

## Fitting the Model

In [6]:
def generator_wrapper(generator):
    for batch_x,batch_y in generator:
        yield (batch_x,[batch_y[:,i] for i in range(num_hashtags)])

STEP_SIZE_TRAIN = train_generator.n//train_generator.batch_size
STEP_SIZE_VALID = valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST = test_generator.n//test_generator.batch_size

model.fit_generator(generator=generator_wrapper(train_generator),
    steps_per_epoch=STEP_SIZE_TRAIN,
    validation_data=generator_wrapper(valid_generator),
    validation_steps=STEP_SIZE_VALID,
    epochs=1,
    verbose=0)

In [6]:
# Save model for future use
model.save(destination("../model", "keras_model"))

## Prediction

In [14]:
test_generator.reset()
pred = model.predict_generator(test_generator, steps=STEP_SIZE_TEST, verbose=1)

5739/5739 [==============================] - 322s 56ms/step


In [180]:
# Zscore normalization
pred_reshaped = np.array(pred).reshape((num_hashtags, num_images-val_idx)).transpose()
normalized = (pred_reshaped - pred_reshaped.mean())/pred_reshaped.std()
above3std = (b > 3).astype(int)

# Get lists of hashtags
y_pred = pd.concat([
    pd.Series(test_generator.filenames, name="filename"),
    pd.Series(vectorizer.inverse_transform(above3std), name="y_pred")],
    axis=1)

y_true = target[target["filename"].isin(results["filename"])][["filename", "labels"]]
y_true["y_true"] = y_true["labels"].apply(lambda x: x.split())
y_true.drop(["labels"], axis=1, inplace=True)

results = pd.merge(y_true, y_pred, left_on="filename", right_on="filename")
results.head()

,filename,y_true,y_pred
0,justinbieber_image_1127.jpg,"[likeme, follow, onedirection, harrystyles, ar...","[black, family, fashion, friend, girl, love, n..."
1,makeup_image_696.jpg,[makeup],"[beach, beautiful, black, bored, family, fashi..."
2,truth_image_2843.jpg,"[word, wisdom, truth, quote]","[beach, beautiful, black, family, fashion, fri..."
3,dog_image_1067.jpg,"[bff, cloud, dog, bestfriends, dogsofinstagram]","[beach, beautiful, beauty, black, bored, boyfr..."
4,family_image_2692.jpg,"[kid, boy, child, park, april, outdoors, famil...","[beach, beautiful, black, bored, family, fashi..."


## Measure Performance
Count the number of hashtags that's not in y_true. Note that this metric is not really useful.

In [191]:
def performance(row):
    err = 0
    for hashtag in  row.y_pred:
        if hashtag not in row.y_true:
            err += 1
    return err

err = results.apply(performance, axis=1)


## Some of the Pictures with predicted hashtags
Only 10 of the predicted hashtags are shown with the pictures.

In [227]:
## Manually Check Some of the Results...
import ipyplot

paths = list(results.filename.apply(lambda x: f"../HARRISON/images/{x}"))
labels = [tags[:10] for tags in results.y_pred]

ipyplot.plot_images(paths, labels)

## Reference
https://medium.com/@vijayabhaskar96/multi-label-image-classification-tutorial-with-keras-imagedatagenerator-cd541f8eaf24